In [1]:
import pandas as pd

In [2]:
cols = ['user', 'item', 'rating', 'timestamp']

In [3]:
train = pd.read_csv('./data/ua.base', delimiter='\t', names=cols)

In [10]:
train[:]

,user,item,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
5,1,6,5,887431973
6,1,7,4,875071561
7,1,8,1,875072484
8,1,9,5,878543541
9,1,10,3,875693118


In [29]:
import numpy as np
from scipy.sparse import csr
import pandas as pd

def vectorize_dic(dic, index=None, p=None, n=0, g=0):
    '''
    :param dic: dictonary of feature lists, keys are the name of features
    :param index:  index generator
    :param p:  dimension of feature space, columns in the sparse matrix
    :param n: number of samples
    :param g:
    :return:
    '''

    if index == None:
        index = dict()

    nz = n * g

    # [1,1,2,1,3,1,4,1...]
    col_ix = np.empty(nz, dtype=int)

    i = 0
    for k, lst in dic.items():
        for t in range(len(lst)):
            index[str(lst[t]) + str(k)] = index.get(str(lst[t]) + str(k), 0) + 1
            col_ix[i + t * g] = index[str(lst[t]) + str(k)]
        i += 1

    row_ix = np.repeat(np.arange(0, n), g)
    data = np.ones(nz)
    if p == None:
        p = len(index)

    index_ = np.where(col_ix < p)
    return csr.csr_matrix((data[index_], (row_ix[index_], col_ix[index_])), shape=(n, p)), index

cols = ['user', 'item', 'rating', 'timestamp']

train = pd.read_csv('./data/ua1.base', delimiter='\t', names=cols)
test = pd.read_csv('./data/ua.test', delimiter='\t', names=cols)

x_train, x_train_index = vectorize_dic({'users': train['user'].values,
                                  'items': train['item'].values}, n=len(train.index), g=2)
x_test, x_test_index = vectorize_dic({'users': test['user'].values,
                                  'items': test['item'].values}, n=len(test.index), g=2)

print('x_train', x_train)
y_train = train['rating'].values
y_test = test['rating'].values

x_train = x_train.todense()
x_test = x_test.todense()

print(x_train)
print(x_train.shape)
print(x_test.shape)

n, p = x_train.shape




x_train   (0, 1)	2.0
  (1, 1)	1.0
  (1, 2)	1.0
  (2, 1)	1.0
  (2, 3)	1.0
  (3, 1)	1.0
  (3, 4)	1.0
  (4, 1)	1.0
  (4, 2)	1.0
[[0. 2. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 1. 1. 0. 0. 0.]]
(5, 6)
(9430, 2072)


In [31]:
x_train

matrix([[0., 2., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0.],
        [0., 1., 1., 0., 0., 0.]])